# Recurrent Neural Network (RNN) for NLP

---

## Outline of Document
1. RNN Model with LSTM
2. Model Cross Validation
3. Evaluation Metrics
4. Pros and Cons of Model

## RNN Model with LSTM



### Download Required Libraries

In [ ]:
!pip install tensorflow
!pip install imblearn
!pip install keras==2.6.0

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import csv, datetime, time, json
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from zipfile import ZipFile
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, TimeDistributed, Dense, Lambda, concatenate, Dropout, BatchNormalization, LSTM
from keras.layers.merge import Concatenate
from keras.layers.embeddings import Embedding
from keras.utils.vis_utils import plot_model
from keras.regularizers import l2
from keras import backend as K
from keras.callbacks import Callback, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imblearn.under_sampling import RandomUnderSampler


### Global Constants

In [ ]:
MAX_QN_LEN = 219
MAX_WORDS = 20000
EMBED_DIM = 300 #GloVe is 300dim
MAX_SEQUENCE = 25
TRAIN_TEST = 0.1
VALIDATION = 0.1
SEED = 42
NUM_EPOCHS = 50
DROPOUT = 0.1
BATCH_SIZE = 32  #Is the default value
REGULARIZER = 0.01
OPTIMIZER = 'adam'

BASE_MODEL_WEIGHTS_FILE = "question_pairs_weights.h5" #Will be saved to a new file if this file didnt exist before
MODEL_WEIGHTS_FILE_MLSTM = "question_pairs_weights_MLSTM.h5"
MODEL_WEIGHTS_FILE_MLSTM_FEATURE = "question_pairs_weights_MLSTM_w_Features.h5"

### Processing Data

In [ ]:
%cd "C:\Users\luciu\Desktop\NUS Lecture Notes\Y3S1\CS3244\Project"

C:\Users\luciu\Desktop\NUS Lecture Notes\Y3S1\CS3244\Project


In [ ]:
train_set = pd.read_csv('train_set_with_features.csv', index_col=[0])

In [ ]:
train_set.head(5)

,id,qid1,qid2,question1,question2,is_duplicate,common_words_ratio,common_tokens_ratio,fuzz_partial_ratio,min_longest_substring,...,common_stop_words_max,common_nouns_max,ratio_len_qn,common_word_ratio_max,common_adjectives_max,fuzz_token_set_ratio,common_words_ratio_min,common_adjectives_min,fuzz_token_sort_ratio,max_longest_substring
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,0.478261,0.076923,100,1.000000,...,0.428571,0.214286,1.160714,0.307692,0.0,100,1.000000,0.0,93,0.861538
1,1,3,4,what is the story of kohinoor koh i noor dia...,what would happen if the indian government sto...,0,0.291667,0.045977,74,0.600000,...,0.200000,0.200000,0.574713,0.172414,0.0,86,0.700000,0.0,63,0.344828
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,0.166667,0.027778,46,0.172414,...,0.142857,0.071429,1.241379,0.236111,0.0,63,0.400000,0.0,63,0.138889
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when 23 24 math is divi...,0,0.000000,0.000000,11,0.040816,...,0.000000,0.000000,0.844828,0.224138,0.0,28,0.000000,0.0,25,0.034483
4,4,9,10,which one dissolve in water quikly sugar salt...,which fish would survive in salt water,0,0.200000,0.026667,55,0.157895,...,0.153846,0.153846,1.973684,0.213333,0.0,67,0.571429,0.0,47,0.080000


In [ ]:
Y_labels = train_set["is_duplicate"]
X_features = train_set.drop("is_duplicate", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_features, Y_labels, test_size=TRAIN_TEST, random_state=SEED)

#print(y_train_full.value_counts())

#undersample = RandomUnderSampler(sampling_strategy='majority')
# undersample = RandomUnderSampler(replacement=True)
# X_train, y_train = undersample.fit_resample(X_train_full, y_train_full)

print(y_train.value_counts())

0    229467
1    134372
Name: is_duplicate, dtype: int64


In [ ]:
X_train_vector = X_train[['question1', 'question2']]
X_train_vector['question2'] = X_train_vector['question2'].astype(str)
X_train_vector.head(5)

C:\Users\luciu\AppData\Local\Temp/ipykernel_9692/2052425777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_vector['question2'] = X_train_vector['question2'].astype(str)


,question1,question2
305968,what are some examples of two kind of people,what are some cool examples of two kinds of pe...
166442,how it would be to join time for cat in februa...,how it would be to join time for cat in february
87947,what is the best way to sell my restaurant,how do i sell a restaurant idea
117082,what will be hillary clinton s foreign policy ...,what will hillary clinton s chinese policy be ...
141242,what food can increase dopamine respectively s...,what are the best supplements to increase dopa...


In [ ]:
X_train_features = X_train.drop(['question1', 'question2', 'qid1', 'qid2', 'id'], axis=1)
X_train_features.head(5)

,common_words_ratio,common_tokens_ratio,fuzz_partial_ratio,min_longest_substring,unique_words_count,common_token_ratio_min,fuzz_ratio,abs_len_difference,common_stop_words_min,common_nouns_min,...,common_stop_words_max,common_nouns_max,ratio_len_qn,common_word_ratio_max,common_adjectives_max,fuzz_token_set_ratio,common_words_ratio_min,common_adjectives_min,fuzz_token_sort_ratio,max_longest_substring
305968,0.411765,0.060000,86,0.477273,7,0.068182,94,6,0.444444,0.000000,...,0.400000,0.000000,0.880000,0.360000,0.000000,94,0.875000,0.000000,94,0.420000
166442,0.478261,0.081967,100,1.000000,11,0.104167,88,13,0.545455,0.181818,...,0.428571,0.142857,1.270833,0.311475,0.016393,100,1.000000,0.020833,88,0.786885
87947,0.125000,0.047619,63,0.354839,2,0.064516,55,11,0.000000,0.142857,...,0.000000,0.111111,1.354839,0.309524,0.000000,65,0.285714,0.000000,52,0.261905
117082,0.423077,0.051948,73,0.296875,11,0.062500,79,13,0.583333,0.166667,...,0.500000,0.142857,1.203125,0.233766,0.000000,93,0.916667,0.000000,86,0.246753
141242,0.235294,0.046875,69,0.351852,4,0.055556,59,10,0.142857,0.142857,...,0.100000,0.100000,0.843750,0.250000,0.000000,75,0.571429,0.000000,59,0.296875


In [ ]:
X_test_vector = X_test[['question1', 'question2']]
X_test_vector['question2'] = X_test_vector['question2'].astype(str)
X_test_vector.head(5)

C:\Users\luciu\AppData\Local\Temp/ipykernel_9692/325267862.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_vector['question2'] = X_test_vector['question2'].astype(str)


,question1,question2
162455,how good a phil barone saxophones,what are phil barone saxophones
34708,why have you converted to islam,why have you converted to islam what is your ...
158538,how do i learn and master things,how can i learn mastering music
61781,can dinosaurs exist on other planets,are there other worlds that have dinosaurs on ...
78367,which is the best movie from 2016,which is the best movie of 2016


In [ ]:
X_test_features = X_test.drop(['question1', 'question2', 'qid1', 'qid2', 'id'], axis=1)
X_test_features.head(5)

,common_words_ratio,common_tokens_ratio,fuzz_partial_ratio,min_longest_substring,unique_words_count,common_token_ratio_min,fuzz_ratio,abs_len_difference,common_stop_words_min,common_nouns_min,...,common_stop_words_max,common_nouns_max,ratio_len_qn,common_word_ratio_max,common_adjectives_max,fuzz_token_set_ratio,common_words_ratio_min,common_adjectives_min,fuzz_token_sort_ratio,max_longest_substring
162455,0.272727,0.090909,84,0.741935,3,0.096774,81,2,0.000000,0.000000,...,0.000000,0.000000,1.064516,0.424242,0.000000,83,0.600000,0.000000,75,0.696970
34708,0.375000,0.039216,100,1.000000,6,0.064516,76,20,0.666667,0.000000,...,0.400000,0.000000,0.607843,0.352941,0.000000,100,1.000000,0.000000,77,0.607843
158538,0.230769,0.031250,71,0.290323,3,0.032258,73,1,0.333333,0.000000,...,0.285714,0.000000,1.032258,0.437500,0.000000,79,0.500000,0.000000,79,0.281250
61781,0.200000,0.020000,63,0.305556,3,0.027778,42,14,0.333333,0.000000,...,0.222222,0.000000,0.720000,0.260000,0.020000,67,0.500000,0.027778,58,0.220000
78367,0.428571,0.090909,90,0.774194,6,0.096774,94,2,0.428571,0.142857,...,0.428571,0.142857,1.064516,0.515152,0.030303,95,0.857143,0.032258,88,0.727273


In [ ]:
X_test_features.columns

Index(['common_words_ratio', 'common_tokens_ratio', 'fuzz_partial_ratio',
       'min_longest_substring', 'unique_words_count', 'common_token_ratio_min',
       'fuzz_ratio', 'abs_len_difference', 'common_stop_words_min',
       'common_nouns_min', 'mean_len', 'common_stop_words_max',
       'common_nouns_max', 'ratio_len_qn', 'common_word_ratio_max',
       'common_adjectives_max', 'fuzz_token_set_ratio',
       'common_words_ratio_min', 'common_adjectives_min',
       'fuzz_token_sort_ratio', 'max_longest_substring'],
      dtype='object')

Tokenizer creates a keras.tokenizer object which will convert the list of strings that a user pushes in into a list of [tokens]. Based on MAX_NUMBER_OF_WORDS, the tokenizer will cut the number of tokens to that size if needed. In this case, since I put 20k as the limit, we will not be cutting anything. 

text_to_sequence fits the list of sentences into list of tokens. 
word_index provides the total number of words in this object

In [ ]:
questions = X_train_vector['question1'].tolist() + X_train_vector['question2'].tolist()
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(questions)
#Training Set
question1_train = tokenizer.texts_to_sequences(X_train_vector['question1'].tolist())
question2_train = tokenizer.texts_to_sequences(X_train_vector['question2'].tolist())

#Testing Set
question1_test = tokenizer.texts_to_sequences(X_test_vector['question1'].tolist())
question2_test = tokenizer.texts_to_sequences(X_test_vector['question2'].tolist())

word_index = tokenizer.word_index

In [ ]:
print("Words in index: %d" % len(word_index))

Words in index: 82370


In [ ]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

glove.840b.300d.txt is a 5gb pretrained word vector file by GloVe. The format of the vectors in the txt file should be tuples of word and value. 
eg. 
(Dog, 0.12345) (Cat, 0.12000) etc. 
The code below splits the strings into tuples then forms a dictionary of dict[word] = value for every word in the pretrained model

In [ ]:
embeddings_index = {}
with open('glove.840b.300d.txt', encoding='utf-8') as f:
  for line in f:
    values = line.split(' ')
    word = values[0]
    embedding = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = embedding

In [ ]:
print('Word embeddings: %d' % len(embeddings_index))

Word embeddings: 2196017


In the function below, The number of words is shrunk to the min of MAX_NUMBER_OF_WORDS vs len(word_index). In this case, the number of words = MAX_NUMBER OF WORDS = 20k

For each word in word_index, we take the first 20k words from the our pretrained corpus
and try to tag each word from word_index (Which are words from our questions)
to the corpus.
This seems funny since we will expect to get gaps in the matrix when
the word does not map to any vector in the first 20k of the corpus.
Hence, when the embedding is done, we print the Null word embeddings (indicating the gap) and we see that it is only 435/20000 which can be accepted.

More tuning can be done here to increase/decrease the number of words

In [ ]:
number_of_words = min(MAX_WORDS, len(word_index))
word_embedding_matrix = np.zeros((number_of_words + 1, EMBED_DIM))
for word, i in word_index.items():
  if i > MAX_WORDS:
    continue
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    word_embedding_matrix[i] = embedding_vector
  
print('Null word embeddings: %d' % np.sum(np.sum(word_embedding_matrix, axis=1) == 0))

Null word embeddings: 440


In [ ]:
np.savetxt("word_embedding.csv", word_embedding_matrix, delimiter=",")

For LSTM, the length of our q1 and q2 has to be the same. Therefore, we pad them to an equal length of MAX_SEQUENCE_LENGTH which has been set to 25

In [ ]:
q1_train_data = pad_sequences(question1_train, maxlen=MAX_SEQUENCE)
q2_train_data = pad_sequences(question2_train, maxlen=MAX_SEQUENCE)

q1_test_data =  pad_sequences(question1_test, maxlen=MAX_SEQUENCE)
q2_test_data =  pad_sequences(question2_test, maxlen=MAX_SEQUENCE)

print('Shape of question1 train data tensor:', q1_train_data.shape)
print('Shape of question2 train data tensor:', q2_train_data.shape)

print('Shape of question1 test data tensor:', q1_test_data.shape)
print('Shape of question2 test data tensor:', q1_test_data.shape)

print('Shape of label tensor:', Y_labels.shape)

Shape of question1 train data tensor: (363839, 25)
Shape of question2 train data tensor: (363839, 25)
Shape of question1 test data tensor: (40427, 25)
Shape of question2 test data tensor: (40427, 25)
Shape of label tensor: (404266,)


## Baseline Model

#### Training the model

In [ ]:
question1_train = Input(shape=(MAX_SEQUENCE,)) 
question2_train = Input(shape=(MAX_SEQUENCE,))  

In [ ]:
question1_test = Input(shape=(MAX_SEQUENCE,))
question2_test = Input(shape=(MAX_SEQUENCE,)) 

In the two functions below, several Keras functions are used. 

Embedding(
  1. input_dim = (size of vocab list (20k),
  2. output_dim = (Length of vector of each word (300),
  3. weights = The pretrained matrix that we are using,
  4. input_length = Our padded length (25),
  5. trainable = False (prevents our pretrained weights from being updated)
  )

This creates an Embedding object which should take in a sequences of integers [basically list of integers (index of the words)], ensuring that the sequences have the same length. The embedding layer then maps the integer inputs to the vectors found in the embedding matrix in the weights. 
This creates a 3D matrix of (samples, sequence_length, embedding_dim)

Dense(
  1. units = Output Dimensions (300)
  2. activation = 'relu' refers to rectified linear unit activation function which helps to speed up training and do some stuff (IDK I will find out soon)
)

This creates an Output object that is shrunk to the matrix size of units, which in this case is EMBEDDING_DIMENSIONS which is 300 (No change)

TimeDistributed(
1. Layer (which for now is Dense(Embedding))
2. q1 (which will be the inputs to feed into this layers for data preparation)

TimeDistributed will feed the inputs of q1 or q2 into the layer which prepares each input with the given layer (Dense(Embedding)). In other words. TimeDistributed Layer will apply the same layer to several inputs and each input will produce 1 output. 

K (keras.backend)
1. x is a tensor or variable
2. axis = 1 refers to the column of the matrix 

K is keras backend (imported as K) which provides a list of ultilites such as Max, Min etc.
In the lambda function, K.max converts each tensor (matrix) into the column of the maximum x value.
(Why is there a need to change to Max of columns though)

Lambda
1. lambda = The conversion from x to max of x
2. output dimensions = expected output shape

This is a layer to implement the lambda conversion of x to max of x. (using K.max). 



In [ ]:
q1 = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(question1_train) 
q1 = TimeDistributed(Dense(EMBED_DIM, activation='relu'))(q1) 
q1 = Lambda(lambda x: K.max(x, axis=1), output_shape=(EMBED_DIM,))(q1) 

In [ ]:
q2 = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(question2_train)
q2 = TimeDistributed(Dense(EMBED_DIM, activation='relu'))(q2)
q2 = Lambda(lambda x: K.max(x, axis=1), output_shape=(EMBED_DIM,))(q2)

Dense (See above)

Dropout
1. droput rate = 0.1 (DROPOUT Global constant)

Dropout Layer randomly sets input units to 0 with a frequency of dropout rate during each step. This is to randomly drop the noise which will help to prevent overfitting. 

BatchNormalization
1. No arguments

BatchNormalization applies a transofrmation that maintains the mean output close to 0 and output standard deviation close to 1. This will help to solve internal covariate shift which happens when training our neural network which results in the change in weights of each neuron. The change in weights will cause the distribution of input data for each layer to be different in each iteration which will slow down learning.
Normalizing the weights will help to keep the mean output to 0 and standard deviation close to 1 which will reduce covariate shift. 


Apply 4 layers of training

In [ ]:
inputs = concatenate([q1, q2])
inputs = Dense(70, activation='relu')(inputs)     #First
inputs = Dropout(DROPOUT)(inputs)
inputs = BatchNormalization()(inputs)
inputs = Dense(150, activation='relu')(inputs)     #Second
inputs = Dropout(DROPOUT)(inputs)
inputs = BatchNormalization()(inputs)
inputs = Dense(70, activation='relu')(inputs)     #Third
inputs = Dropout(DROPOUT)(inputs)
inputs = BatchNormalization()(inputs)
inputs = Dense(35, activation='relu')(inputs)     #Fourth
inputs = Dropout(DROPOUT)(inputs)
inputs = BatchNormalization()(inputs)

Create a compressed output of each tensor as single value output using the Dense layer(1...) which means 1 dimension eg 1 value
Sigmoid is used for logistic regression to linearly classify the output. 

In [ ]:
final_output = Dense(1, activation='sigmoid')(inputs)

In [ ]:
model = Model(inputs=[question1_train, question2_train], outputs=final_output)
model.compile(loss='binary_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 25, 300)      6000300     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 25, 300)      6000300     input_2[0][0]                    
______________________________________________________________________________________________

Using the prepared layers, now feed in the training questions and their original labels.

In [ ]:
print("Starting training at", datetime.datetime.now())
t0 = time.time()
callbacks = [ModelCheckpoint(MODEL_WEIGHTS_FILE, monitor='val_accuracy', save_best_only=True)]
history = model.fit([q1_train_data, q2_train_data], 
                    y_train, 
                    epochs=NUM_EPOCHS,
                    validation_split=VALIDATION,
                    verbose=1,
                    batch_size=BATCH_SIZE,
                    callbacks=callbacks)

t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

Starting training at 2021-11-27 15:50:18.348193
10233/10233 [==============================] - 109s 10ms/step - loss: 0.5415 - accuracy: 0.7253 - val_loss: 0.4959 - val_accuracy: 0.7527
Training ended at 2021-11-27 15:52:08.420747
Minutes elapsed: 1.834526


In [ ]:
max_val_acc, idx = max((val, idx) for (idx, val) in enumerate(history.history['val_accuracy']))
print('Maximum validation accuracy = {0:.4f} (epoch {1:d})'.format(max_val_acc, idx+1))

#### Testing the model

In [ ]:
model.load_weights(BASE_MODEL_WEIGHTS_FILE)
loss, accuracy = model.evaluate([q1_test_data, q2_test_data], y_test, verbose=0)
print('Test loss = {0:.4f}, test accuracy = {1:.4f}'.format(loss, accuracy))

####  Evaluation Metrics

In [ ]:
print(history.history['accuracy'])
print(history.history['loss'])
print(history.history['val_accuracy'])
print(history.history['val_loss'])

In [ ]:
plt.plot(history.history['loss'], color ='blue');
plt.plot(history.history['val_loss'], color='red');
plt.title('Training Loss Vs Validation Loss');
plt.xlabel('Epochs');
plt.ylabel('Loss');

In [ ]:
plt.plot(history.history['accuracy'], color ='blue');
plt.plot(history.history['val_accuracy'], color='red');
plt.title('Training Accuracy Vs Validation Accuracy');
plt.xlabel('Epochs');
plt.ylabel('Accuracy');

In [ ]:
incorrects = model.predict([q1_test_data, q2_test_data], verbose=1)

In [ ]:
incorrects[incorrects > 0.5] = 1
incorrects[incorrects <= 0.5] = 0
flattened_incorrect = incorrects.flatten()

In [ ]:
heatmap_data = confusion_matrix(y_test, flattened_incorrect)
group_names = ["True Neg","False Pos","False Neg","True Pos"]
group_counts = ["{0}".format(value) for value in heatmap_data.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in heatmap_data.flatten()/np.sum(heatmap_data)]
labels = [f"{group_name}\n{group_count}\n{group_percentage}" for group_name, group_count, group_percentage in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(heatmap_data, annot=labels, fmt="", cmap='Blues')

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, flattened_incorrect).ravel()
Precision = tp / (tp + fp)
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Recall = tp / (tp + fn)
F1 = 2 / ((1/Recall) + (1/Precision))

print("Precision: " + str(Precision) + "\n" + "Accuracy: " + str(Accuracy) + "\n" + "Recall: " + str(Recall) + "\n" + "F1: " + str(F1))

In [ ]:
model.save("BASELINE_MODEL.h5")

## Using Siamnese LSTM with Manhatten Distance


#### Training the Model

In [ ]:
# NEW MODEL TRYING TO USE SIAMNESE LSTM
def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))


q1_input = Input(shape=(MAX_SEQUENCE,))
q2_input = Input(shape=(MAX_SEQUENCE,))

LSTM_DIM = 128
LSTM_DROPOUT = 0.10
LSTM_REGULARIZATION = 0.00

q1_LSTM = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(q1_input)
q2_LSTM = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(q2_input)

manhatten_LSTM = LSTM(LSTM_DIM, kernel_regularizer=l2(LSTM_REGULARIZATION), dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
q1_LSTM_Output = manhatten_LSTM(q1_LSTM)
q2_LSTM_Output = manhatten_LSTM(q2_LSTM)

malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([q1_LSTM_Output, q2_LSTM_Output])


malstm = Model([q1_input, q2_input], [malstm_distance])
malstm.compile(loss='mean_squared_error', optimizer=OPTIMIZER, metrics=['accuracy'])
malstm.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 25, 300)      6000300     input_5[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 25, 300)      6000300     input_6[0][0]                    
____________________________________________________________________________________________

In [ ]:
print("Starting training at", datetime.datetime.now())
t0 = time.time()

callbacks_MLSTM = [ModelCheckpoint(MODEL_WEIGHTS_FILE_MLSTM, monitor='val_accuracy', save_best_only=True)]
malstm_trained = malstm.fit([q1_train_data, q2_train_data],
                            y_train,
                            batch_size=BATCH_SIZE,
                            epochs=NUM_EPOCHS,
                            validation_split=VALIDATION,
                            verbose = 1,
                            callbacks = callbacks_MLSTM)

t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

In [ ]:
max_val_acc, idx = max((val, idx) for (idx, val) in enumerate(malstm_trained.history['val_accuracy']))
print('Maximum validation accuracy = {0:.4f} (epoch {1:d})'.format(max_val_acc, idx+1))

#### Testing the Model

In [ ]:
malstm.load_weights(MODEL_WEIGHTS_FILE_MLSTM)
loss, accuracy = malstm.evaluate([q1_test_data, q2_test_data], y_test, verbose=1)
print('Test loss = {0:.4f}, test accuracy = {1:.4f}'.format(loss, accuracy))

#### Evaluation Metrics

In [ ]:
plt.plot(malstm_trained.history['loss'], color ='blue');
plt.plot(malstm_trained.history['val_loss'], color='red');
plt.title('Training Loss Vs Validation Loss');
plt.xlabel('Epochs');
plt.ylabel('Loss');

In [ ]:
plt.plot(malstm_trained.history['accuracy'], color ='blue');
plt.plot(malstm_trained.history['val_accuracy'], color='red');
plt.title('Training Accuracy Vs Validation Accuracy');
plt.xlabel('Epochs');
plt.ylabel('Accuracy');

In [ ]:
incorrects = malstm.predict([q1_test_data, q2_test_data], verbose=1)

In [ ]:
incorrects[incorrects > 0.5] = 1
incorrects[incorrects <= 0.5] = 0
flattened_incorrect = incorrects.flatten()

In [ ]:
heatmap_data = confusion_matrix(y_test, flattened_incorrect)
group_names = ["True Neg","False Pos","False Neg","True Pos"]
group_counts = ["{0}".format(value) for value in heatmap_data.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in heatmap_data.flatten()/np.sum(heatmap_data)]
labels = [f"{name}\n{count}\n{percentage}" for name, count, percentage in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(heatmap_data, annot=labels, fmt="", cmap='Blues')

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, flattened_incorrect).ravel()
Precision = tp / (tp + fp)
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Recall = tp / (tp + fn)
F1 = 2 / ((1/Recall) + (1/Precision))

print(f"Precision: {Precision} \nAccuracy: {Accuracy} \nRecall: {Recall} \nF1: {F1}")

In [ ]:
malstm.save("MALTSM_No_Features.h5")

## Using Siamnese LSTM with Features

####Training the Model

In [ ]:
def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))


q1_input = Input(shape=(MAX_SEQUENCE,))  #25-dim vectors 
q2_input = Input(shape=(MAX_SEQUENCE,))  #25-dim vectors

LSTM_DIM = 128
LSTM_DROPOUT = 0.10
LSTM_REGULARIZATION = 0.0001

q1_LSTM = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(q1_input)  #Creates an Embedding Object and feeds question1 into it
q2_LSTM = Embedding(number_of_words + 1,
               EMBED_DIM,
               weights=[word_embedding_matrix],
               input_length=MAX_SEQUENCE,
               trainable=False)(q2_input)


manhatten_LSTM = LSTM(LSTM_DIM, kernel_regularizer=l2(LSTM_REGULARIZATION), dropout=LSTM_DROPOUT, recurrent_dropout=LSTM_DROPOUT)
q1_LSTM_Output = manhatten_LSTM(q1_LSTM)
q2_LSTM_Output = manhatten_LSTM(q2_LSTM)

malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([q1_LSTM_Output, q2_LSTM_Output])  #What is x[0][0]??

no_of_features = len(X_train_features.columns)
feature_input = Input(shape=(no_of_features,))
feature_layer = Dense(70, activation='relu')(feature_input)

concat_layer = Concatenate()([q1_LSTM_Output, q2_LSTM_Output, feature_layer])
concat_layer = Dropout(DROPOUT)(concat_layer)
concat_layer = BatchNormalization()(concat_layer)

concat_layer = Dense(150, kernel_regularizer=l2(REGULARIZER), activation='relu')(concat_layer)
concat_layer = Dropout(DROPOUT)(concat_layer)
concat_layer = BatchNormalization()(concat_layer)

concat_layer = Dense(70, kernel_regularizer=l2(REGULARIZER), activation='relu')(concat_layer)
concat_layer = Dropout(DROPOUT)(concat_layer)
concat_layer = BatchNormalization()(concat_layer)

concat_layer = Dense(35, kernel_regularizer=l2(REGULARIZER), activation='relu')(concat_layer)
concat_layer = Dropout(DROPOUT)(concat_layer)
concat_layer = BatchNormalization()(concat_layer)

output = Dense(1, activation='sigmoid')(concat_layer)

malstm_w_feature = Model([q1_input, q2_input, feature_input], [output])
malstm_w_feature.compile(loss='binary_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'])
malstm_w_feature.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 25)]         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 25, 300)      6000300     input_7[0][0]                    
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 25, 300)      6000300     input_8[0][0]                    
____________________________________________________________________________________________

In [ ]:
print("Starting training at", datetime.datetime.now())
t0 = time.time()

callbacks_MLSTM_feature = [ModelCheckpoint(MODEL_WEIGHTS_FILE_MLSTM_FEATURE, monitor='val_accuracy', save_best_only=True)]
malstm_w_feature_trained = malstm_w_feature.fit([q1_train_data, q2_train_data, X_train_features],
                            y_train,
                            batch_size=BATCH_SIZE,
                            epochs=NUM_EPOCHS,
                            validation_split=VALIDATION,
                            verbose = 1,
                            callbacks = callbacks_MLSTM_feature)

t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

In [ ]:
max_val_acc, idx = max((val, idx) for (idx, val) in enumerate(malstm_w_feature_trained.history['val_accuracy']))
print('Maximum validation accuracy = {0:.4f} (epoch {1:d})'.format(max_val_acc, idx+1))

####Testing the Model

In [ ]:
malstm_w_feature.load_weights(MODEL_WEIGHTS_FILE_MLSTM_FEATURE)
loss, accuracy = malstm_w_feature.evaluate([q1_test_data, q2_test_data, X_test_features], y_test, verbose=1)
print('Test loss = {0:.4f}, test accuracy = {1:.4f}'.format(loss, accuracy))

### Evaluation Metrics

In [ ]:
incorrects = malstm_w_feature.predict([q1_test_data, q2_test_data, X_test_features], verbose=1)

In [ ]:
incorrects[incorrects > 0.5] = 1
incorrects[incorrects <= 0.5] = 0
flattened_incorrect = incorrects.flatten()

In [ ]:
heatmap_data = confusion_matrix(y_test, flattened_incorrect)
group_names = ["True Neg","False Pos","False Neg","True Pos"]
group_counts = ["{0}".format(value) for value in heatmap_data.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in heatmap_data.flatten()/np.sum(heatmap_data)]
labels = [f"{name}\n{count}\n{percentage}" for name, count, percentage in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(heatmap_data, annot=labels, fmt="", cmap='Blues')

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, flattened_incorrect).ravel()
Precision = tp / (tp + fp)
Accuracy = (tp + tn) / (tp + tn + fp + fn)
Recall = tp / (tp + fn)
F1 = 2 / ((1/Recall) + (1/Precision))

print(f"Precision: {Precision} \nAccuracy: {Accuracy} \nRecall: {Recall} \nF1: {F1}")

In [ ]:
malstm_w_feature.save("MALTSM_w_features.h5")